In [3]:
import csv
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train_data=[]
target=[]
with open('train.csv') as dat:
    dat.readline()
    reader = csv.reader(dat, delimiter = ',')
    for row in reader:
        temp=cv2.imread(os.path.join(os.getcwd(), 'train', row[0]))
        temp1=cv2.resize(temp, (100,100))
        train_data.append(temp1)
        target.append(row[1])
        


In [5]:
train_data=np.array(train_data)
train_data=train_data/255.
target=np.array(target)
uni=np.unique(target)

In [6]:

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = target.reshape(len(target), 1)
Y = onehot_encoder.fit_transform(integer_encoded)


In [7]:
test_data=[]

with open('test_WyRytb0.csv') as dat:
    dat.readline()
    reader = csv.reader(dat, delimiter = ',')
    for row in reader:
        temp=cv2.imread(os.path.join(os.getcwd(), 'train', row[0]))
        temp1=cv2.resize(temp, (100,100))
        test_data.append(temp1)
        
        
test_data=np.array(test_data)
print(test_data.shape)

(7301, 100, 100, 3)


In [26]:
def IntelNet(input_shape = (100, 100, 3), classes = len(uni)):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(128, (7, 7), strides = (2, 2), name = 'conv1',padding = 'SAME')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2),padding = 'SAME')(X)
    

    X = Conv2D(128, (5, 5), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    X = Conv2D(128, (5, 5), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3, 3), strides = (1, 1), name = 'conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv4')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    
    
    
    
    
    
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [27]:
model = IntelNet(input_shape = (100, 100, 3), classes = len(uni))

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(train_data,Y,test_size = 0.2)

In [ ]:
model.fit(train_data, Y, epochs = 100, batch_size = 100)

Epoch 1/100
17034/17034 [==============================] - 139s 8ms/step - loss: 13.1769 - acc: 0.1776
Epoch 2/100
17034/17034 [==============================] - 118s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 3/100
17034/17034 [==============================] - 113s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 4/100
17034/17034 [==============================] - 112s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 5/100
17034/17034 [==============================] - 113s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 6/100
17034/17034 [==============================] - 111s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 7/100
17034/17034 [==============================] - 112s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 8/100
17034/17034 [==============================] - 114s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 9/100
17034/17034 [==============================] - 113s 7ms/step - loss: 13.2444 - acc: 0.1783
Epoch 10/100
16000/17034 [===========================>..] - ETA: 6s - los

In [ ]:
probs=model.predict(test_data)
classe=probs.argmax(axis=-1)
print(classe)

In [ ]:
import pandas as pd
final_data=[]
i=0
with open('test_WyRytb0.csv') as dat:
    dat.readline()
    reader = csv.reader(dat, delimiter = ',')
    for row in reader:
        final_data.append([row[0], classe[i]])
        i=i+1
final_dataframe=pd.DataFrame(final_data,columns = ['image_name','label'])
final_dataframe.to_csv('dl_result.csv')
        